In [1]:
import os
import numpy as np
#from proxy_env import RLBot
import matplotlib.pyplot as plt
import pickle
from keras.models import load_model, Sequential
from keras.layers import Dense, Activation
from keras.utils.vis_utils import plot_model
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

/home/daniel/anaconda2/envs/tensorflow/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
observations = {}
observations['proxy_sensor'] = [0.1,0.2,0.3,0.4,3,4,5,6,0.001,0,0,1,0.001,4,5,0]
observations['light_sensor'] = []
observations['proxy_sensor'] = np.asarray(observations['proxy_sensor'])
observations['proxy_sensor'][observations['proxy_sensor'] < 0.001] = 2
print observations['proxy_sensor'].shape


(16,)


In [3]:
reward_robot = {}
reward_robot['proxy_sensor'] = (observations['proxy_sensor'] < 0.7).sum() * -2
reward_robot['proxy_sensor'] += (observations['proxy_sensor'] == 0).sum() * -10
print reward_robot
r = np.clip(np.sum(np.absolute([0.8,0.8])) * 2, 0, 2)
reward_robot['proxy_sensor'] += r
print reward_robot

{'proxy_sensor': -12}
{'proxy_sensor': -10.0}


In [4]:
def train():
    #Initializing model
    print 'Building Model.'
    
    try:
        model = load_model('model_porxy.hdf5')
        
        print 'Saved model found.'

    except:
        print "Model Created"
        model = Sequential()
        model.add(Dense(units=10, input_dim=16))
        model.add(Activation("relu"))
        model.add(Dense(units=12))
        model.add(Activation("relu"))
        model.add(Dense(units=5))
        model.add(Activation("relu"))
        model.compile(optimizer='Adam', loss='categorical_crossentropy')
        print 'No model found, creating new.'
        plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
            
    #Initializing environment
    #env = RLBot()

    # Set learning parameters
    y = .99
    e = 0.1
    num_epochs = 10
    num_steps = 300
    # create lists to contain total rewards and steps per episode
    stepList = []
    rewardList = []
    lossList = []
    SPEED = 0.8

    for i in range(num_epochs):
        # Reset environment and get first new observation
        #env.reset()
        state, reward = observations, reward_robot
        
        state = state['proxy_sensor'].reshape((1, -1))
        
        reward = reward['proxy_sensor']
        Q = model.predict(state)
        action = Q.argmax()
        rAll = 0
        done = False
        loss = 0
        # The Q-Network"""
        for j in range(num_steps):
            print("Step {} | Action: {} | Reward: {}".format(j, action, reward))
            # Choose an action by greedily (with e chance of random action)
            # from the Q-network
            Q = model.predict(state)
            print Q
            action = Q.argmax()
           
            # random action
            if np.random.rand(1) < e:
                action = 5
                print("e = {}. Choosing Random Action: {}".format(e, action))
            # Get new state and reward from environment
            speed = np.zeros(2)
            # Q -> left, forward, backward, break
            if action == 0:
                speed[0] = 0
                speed[1] = SPEED
            if action == 1:
                speed[0] = -SPEED
                speed[1] = -SPEED
            if action == 2:
                speed[0] = SPEED
                speed[1] = SPEED
            if action == 3:
                speed[0] = 0
                speed[1] = 0
            if action == 4:
                speed[0] = SPEED
                speed[1] = 0

            state_, reward_ = observations, reward_robot
            state_ = state_['proxy_sensor'].reshape((1, -1))
            reward_ = reward_['proxy_sensor']
            # Obtain the Q' values by feeding the new state through our network
            Q_ = model.predict(state_)
            # Obtain maxQ' and set our target value for chosen action.
            maxQ_ = np.max(Q_)
            targetQ = Q
            
            action = action-1
            
            targetQ[0, action] = reward + y * maxQ_
            print targetQ.shape
            # Train our network using target and predicted Q values
            loss += model.train_on_batch(state, targetQ)
            rAll += reward
            state = state_
            reward = reward_
            if done is True:
                break
                
        # Reduce chance of random action as we train the model.
        if num_epochs <= 5:
            e -= 0.01 #0.01
            
        else:
            e -= 0.001 #0.002
        stepList.append(j)
        rewardList.append(rAll)
        lossList.append(loss)
        print("Episode: " + str(i))
        print("Loss: " + str(loss))
        print("e: " + str(e))
        print("Reward: " + str(rAll))
        pickle.dump({'stepList': stepList, 'rewardList': rewardList, 'lossList': lossList},
                    open("history_porxy_conf2.p", "wb"))
        model.save('model_proxy_conf2.hdf5')
        
    print("Average loss: " + str(sum(lossList) / num_episodes))
    print("Average number of steps: " + str(sum(stepList) / num_episodes))
    print("Average reward: " + str(sum(rewardList) / num_episodes))

    plt.plot(rewardList)
    plt.plot(stepList)
    plt.plot(lossList)
    return i

if __name__ == '__main__':
    try:
        train()
    except KeyboardInterrupt or i == 100:
        print('Exiting.')


Building Model.
Model Created
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
No model found, creating new.
Step 0 | Action: 2 | Reward: -10.0
[[0.        0.        1.8956819 0.        1.0039833]]
e = 0.1. Choosing Random Action: 5
(1, 5)
Step 1 | Action: 4 | Reward: -10.0
[[0.        0.        1.9049554 0.        0.9513419]]
(1, 5)
Step 2 | Action: 1 | Reward: -10.0
[[0.         0.         1.9120208  0.         0.91079396]]
(1, 5)
Step 3 | Action: 1 | Reward: -10.0
[[0.        0.        1.9177414 0.        0.8776733]]
(1, 5)
Step 4 | Action: 1 | Reward: -10.0
[[0.         0.         1.9225299  0.         0.84974444]]
(1, 5)
Step 5 | Action: 1 | Reward: -10.0
[[0.        0.        1.926622  0.        0.8257218]]
(1, 5)
Step 6 | Action: 1 | Reward: -10.0
[[0.         0.         1.9301697  0.         0.80477893]]
(1, 5)
Step 7 | Action: 1 | Reward: -10.0
[[0.         0.         1.9332769  0.

(1, 5)
Step 120 | Action: 1 | Reward: -10.0
[[0.       0.       2.031204 0.       0.      ]]
(1, 5)
Step 121 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312045 0.        0.       ]]
e = 0.1. Choosing Random Action: 5
(1, 5)
Step 122 | Action: 4 | Reward: -10.0
[[0.        0.        2.0312042 0.        0.       ]]
(1, 5)
Step 123 | Action: 1 | Reward: -10.0
[[0.       0.       2.031205 0.       0.      ]]
(1, 5)
Step 124 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312052 0.        0.       ]]
e = 0.1. Choosing Random Action: 5
(1, 5)
Step 125 | Action: 4 | Reward: -10.0
[[0.        0.        2.0312054 0.        0.       ]]
(1, 5)
Step 126 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312057 0.        0.       ]]
(1, 5)
Step 127 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312057 0.        0.       ]]
(1, 5)
Step 128 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312057 0.        0.       ]]
(1, 5)
Step 129 | Action: 1 | Reward: -10.0
[[0.        0.

Step 236 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 237 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 238 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 239 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 240 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 241 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 242 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 243 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 244 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 245 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 246 | Action: 1

(1, 5)
Step 42 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 43 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
e = 0.099. Choosing Random Action: 5
(1, 5)
Step 44 | Action: 4 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 45 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 46 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 47 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 48 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 49 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 50 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 51 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.   

[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 155 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 156 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 157 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 158 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 159 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 160 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 161 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
e = 0.099. Choosing Random Action: 5
(1, 5)
Step 162 | Action: 4 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 163 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 164 | Action: 1

...

(1, 5)
Step 123 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 124 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 125 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 126 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 127 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
e = 0.093. Choosing Random Action: 5
(1, 5)
Step 128 | Action: 4 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 129 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 130 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 131 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 132 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.   

Step 246 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 247 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 248 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 249 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 250 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 251 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 252 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 253 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 254 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
e = 0.093. Choosing Random Action: 5
(1, 5)
Step 255 | Action: 4 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.

[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 63 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 64 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
e = 0.092. Choosing Random Action: 5
(1, 5)
Step 65 | Action: 4 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 66 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 67 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 68 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 69 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 70 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 71 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 72 | Action: 1 | Reward:

[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 184 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 185 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 186 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 187 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 188 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 189 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 190 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 191 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 192 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 193 | Action: 1 | Reward: -10.0
[[0.        0.      

Step 3 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 4 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 5 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 6 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 7 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 8 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 9 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 10 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
Step 11 | Action: 1 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
e = 0.091. Choosing Random Action: 5
(1, 5)
Step 12 | Action: 4 | Reward: -10.0
[[0.        0.        2.0312066 0.        0.       ]]
(1, 5)
